<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EC%9D%B4%EC%9A%A9%EB%AF%BC_400_fine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import re
import torch
from tqdm.auto import tqdm
import random
import os
from tqdm.notebook import tqdm

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/프로젝트/final project/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

df = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0905_clean.csv")
df.info()

Mounted at /content/drive
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24998 entries, 0 to 24997
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   review  24998 non-null  object 
 1   reply   24998 non-null  object 
 2   star    24998 non-null  int64  
 3   star_t  24998 non-null  int64  
 4   star_q  24998 non-null  int64  
 5   star_d  24998 non-null  float64
 6   food    23986 non-null  object 
 7   store   24998 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 1.5+ MB


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.1 MB/s eta 0:00:00


In [ ]:
!pip install sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.1 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

# 모델과 토크나이저 초기화
model_name = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"
model = AutoModelForSequenceClassification.from_pretrained(model_name).to('cuda')
tokenizer = AutoTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
# 제로샷 분류 파이프라인 초기화
classifier = pipeline("zero-shot-classification", model=model, tokenizer=tokenizer, device=0)  # GPU 사용 설정

# 분류를 위한 레이블 목록
candidate_labels = ['맛', '양', '가격', '배달']

# 전체 리뷰에 대해 제로샷 분류 수행
labels_collected = []
scores_collected = []

for review in tqdm(df['review']):  # 전체 데이터 처리
    output = classifier(review, candidate_labels, multi_label=True)
    labels = [label for label, score in zip(output['labels'], output['scores']) if score > 0.5]
    scores = [round(score, 2) for score in output['scores']]

    labels_collected.append(', '.join(labels))
    scores_collected.append(', '.join([f"{label}: {round(score, 2)}" for label, score in zip(output['labels'], output['scores'])]))

# 결과를 데이터프레임으로 변환
result_df = pd.DataFrame({
    'Review': df['review'],
    'Label': labels_collected,
    'Label-Scores': scores_collected
})

# 데이터프레임을 CSV로 저장

result_df.to_csv(DATA_PATH + "processed_data.csv", index=False)


  0%|          | 0/24998 [00:00<?, ?it/s]

In [ ]:
tokenizer?

In [ ]:
result_df = pd.read_csv(f"{DATA_PATH}processed_data.csv")

In [ ]:
result_df

,Review,Label,Label-Scores
0,저도 모르게 은근 여기 자주 시켜먹고 있었네요 늘 잘먹고 있습니다,"맛, 배달, 가격","맛: 0.98, 배달: 0.89, 가격: 0.8, 양: 0.25"
1,김치찌개 된장찌개 잡채 다 잘먹었습니다,맛,"맛: 0.97, 배달: 0.05, 양: 0.02, 가격: 0.01"
2,처음시켜보는데 맛있고 리뷰서비스도 괜찮네요 다음에도 또시킬듯요,맛,"맛: 0.98, 양: 0.06, 배달: 0.01, 가격: 0.0"
3,맛있습니다 근데 국물이 안와서 아쉽습니다,"맛, 배달","맛: 1.0, 배달: 0.63, 양: 0.23, 가격: 0.19"
4,맛있게잘먹었습니다,맛,"맛: 0.98, 양: 0.2, 배달: 0.05, 가격: 0.0"
...,...,...,...
24993,먹을만하다고느낍니다,"맛, 양","맛: 0.99, 양: 0.6, 가격: 0.36, 배달: 0.02"
24994,조리를해야하는 불편함이 있긴했지만 매장에서 먹는맛 그대로이고 양도 엄청많네요 4인분...,"배달, 맛, 양","배달: 0.97, 맛: 0.87, 양: 0.86, 가격: 0.06"
24995,오랜만에 놀부부대찌게 맛있었는더 햄 양을 늘리심 좋을듯,"맛, 양","맛: 0.99, 양: 0.91, 배달: 0.09, 가격: 0.04"
24996,예전에 참전한 6 25 에서 먹은 부대찌개 맛이 나네요 맛있읍니다,맛,"맛: 1.0, 배달: 0.04, 양: 0.03, 가격: 0.01"


In [ ]:
result_df = result_df.dropna().reset_index(drop=True)

In [ ]:
result_df

,Review,Label,Label-Scores
0,저도 모르게 은근 여기 자주 시켜먹고 있었네요 늘 잘먹고 있습니다,"맛, 배달, 가격","맛: 0.98, 배달: 0.89, 가격: 0.8, 양: 0.25"
1,김치찌개 된장찌개 잡채 다 잘먹었습니다,맛,"맛: 0.97, 배달: 0.05, 양: 0.02, 가격: 0.01"
2,처음시켜보는데 맛있고 리뷰서비스도 괜찮네요 다음에도 또시킬듯요,맛,"맛: 0.98, 양: 0.06, 배달: 0.01, 가격: 0.0"
3,맛있습니다 근데 국물이 안와서 아쉽습니다,"맛, 배달","맛: 1.0, 배달: 0.63, 양: 0.23, 가격: 0.19"
4,맛있게잘먹었습니다,맛,"맛: 0.98, 양: 0.2, 배달: 0.05, 가격: 0.0"
...,...,...,...
24250,먹을만하다고느낍니다,"맛, 양","맛: 0.99, 양: 0.6, 가격: 0.36, 배달: 0.02"
24251,조리를해야하는 불편함이 있긴했지만 매장에서 먹는맛 그대로이고 양도 엄청많네요 4인분...,"배달, 맛, 양","배달: 0.97, 맛: 0.87, 양: 0.86, 가격: 0.06"
24252,오랜만에 놀부부대찌게 맛있었는더 햄 양을 늘리심 좋을듯,"맛, 양","맛: 0.99, 양: 0.91, 배달: 0.09, 가격: 0.04"
24253,예전에 참전한 6 25 에서 먹은 부대찌개 맛이 나네요 맛있읍니다,맛,"맛: 1.0, 배달: 0.04, 양: 0.03, 가격: 0.01"


In [ ]:
re_1000 = result_df[:400]
re_1001 = result_df[1000:2000]

In [ ]:
# # 결과 데이터프레임으로 변환
# result_df = pd.DataFrame({
#     'Review': df['review'][:1000],
#     'Label': labels_collected,
#     'Label-Scores': scores_collected
# })

In [ ]:
from sklearn.model_selection import train_test_split

# 라벨 수정 후 dict 저장
corrections = {
    0: ['맛', '배달'],
    7: ['맛'],
    10: ['맛'],
    18: ['맛', '양'],
    21: ['맛'],
    24 : ['양'],
    28: ['맛'],
    39 : ['양'],
    41: ['맛', '배달'],
    44: ['맛' , '배달'],
    48: ['맛'],
    59: ['맛'],
    65: ['배달', '맛'],
    73: ['맛', '양'],
    74: ['맛', '양'],
    76: ['배달','맛','양'],
    87: ['맛', '양'],
    89: ['맛'],
    92 : ['맛'],
    101 : ['맛', '배달'],
    103 :['맛', '양'],
    106 : ['맛', '양'],
    107 : ['맛', '양'],
    109 : ['배달', '맛'],
    110 : ['맛', '양'],
    124 : ['맛', '양',],
    129 : ['양'],
    154 : ['맛', '양', '가격'],
    155 : ['양', '배달', '맛'],
    162 : ['양', '맛'],
    164 : ['맛', '배달'],
    165 : ['맛'],
    171 : ['맛', '양'],
    186 : ['배달'],
    193 : ['맛'],
    198 : ['맛'],
    210 : ['맛'],
    217 : ['맛'],
    221 : ['맛','양'],
    224 : ['맛'],
    225 : ['배달', '맛'],
    226 : ['가격', '맛'],
    236 : ['가격', '맛'],
    239 : ['맛'],
    244 : ['맛'],
    245 : ['배달'],
    251 : ['맛'],
    254 : ['맛'],
    264 : ['맛', '양'],
    265 : ['맛'],
    268 : ['배달', '맛'],
    269 : ['배'],
    281 : ['맛'],
    297 : ['맛'],
    306 : ['맛', '배달'],
    307 : ['배달', '맛'],
    313 : ['맛'],
    330 : ['맛'],
    333 : ['맛'],
    335 : ['맛'],
    336 : ['배달', '양', '맛'],
    339 : ['배달', '맛'],
    341 : ['맛', '배달'],
    342 : ['맛', '배'],
    356 : ['맛'],
    359 : ['배달', '맛'],
    364 : ['맛', '양'],
    365 : ['양', '맛'],
    366 : ['맛', '배달'],
    368 : ['맛'],
    374 : ['맛', '양'],
    379 : ['배달'],
    381 : ['맛', '양'],
    385 : ['맛'],
    386 : ['배달', '맛'],
    388 : ['맛', '양'],
    392 : ['양', '맛'],
    394 : ['맛', '양'],
    396 : ['맛', '양']
}

# 해당 딕셔너리를 사용하여 라벨을 수정
for index, label in corrections.items():
    re_1000.at[index, 'Label'] = label



In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 3.4 MB/s eta 0:00:00


In [ ]:
re_1000

,Review,Label,Label-Scores
0,저도 모르게 은근 여기 자주 시켜먹고 있었네요 늘 잘먹고 있습니다,"[맛, 배달]","맛: 0.98, 배달: 0.89, 가격: 0.8, 양: 0.25"
1,김치찌개 된장찌개 잡채 다 잘먹었습니다,맛,"맛: 0.97, 배달: 0.05, 양: 0.02, 가격: 0.01"
2,처음시켜보는데 맛있고 리뷰서비스도 괜찮네요 다음에도 또시킬듯요,맛,"맛: 0.98, 양: 0.06, 배달: 0.01, 가격: 0.0"
3,맛있습니다 근데 국물이 안와서 아쉽습니다,"맛, 배달","맛: 1.0, 배달: 0.63, 양: 0.23, 가격: 0.19"
4,맛있게잘먹었습니다,맛,"맛: 0.98, 양: 0.2, 배달: 0.05, 가격: 0.0"
...,...,...,...
395,맛있다는 소문 듣고 드디어 주문해봤어요 엽떡보다 덜 자극적이면서 구성은 비슷해서 정...,"배달, 맛","배달: 0.93, 맛: 0.9, 양: 0.17, 가격: 0.08"
396,실속세트가있어서좋은거같아요 이렇게세트로시키니까 둘이서먹기딱좋은거같네요 남기지않고다먹...,"[맛, 양]","맛: 0.98, 배달: 0.67, 양: 0.28, 가격: 0.19"
397,배달은 항상 50분 넘게걸려요 그래도 맛있으니 2번째요,"맛, 배달","맛: 0.99, 배달: 0.99, 가격: 0.33, 양: 0.1"
398,떡깨비는 첨 시켜보는 곳인데 너무 좋아요 개인적인 요청사항이 있었어서 전화로 한번 ...,맛,"맛: 0.79, 양: 0.26, 배달: 0.21, 가격: 0.13"


In [ ]:
import pandas as pd
import numpy as np

# 레이블을 숫자로 변환하는 딕셔너리
label_to_num = {'맛': 0, '양': 1, '가격': 2, '배달': 3}

# 문자열 레이블을 정렬된 리스트로 변환하는 함수
def string_to_list(label_str):
    if isinstance(label_str, str):
        labels = [label.strip() for label in label_str.split(',')]
        return sorted(labels)  # 레이블 정렬
    return label_str

# 수정된 labels_to_onehot 함수
def labels_to_onehot(labels, label_to_num):
    onehot = [0]*len(label_to_num)
    for label in labels:
        if label in label_to_num:
            onehot[label_to_num[label]] = 1
    return onehot

# 문자열 레이블을 리스트로 변환
re_1000['Label_List'] = re_1000['Label'].apply(string_to_list)
re_1001['Label_List'] = re_1001['Label'].apply(string_to_list)

# 리스트 형태의 레이블을 one-hot encoding으로 변환
re_1000['OneHot_Labels'] = re_1000['Label_List'].apply(lambda x: labels_to_onehot(x, label_to_num))
re_1001['OneHot_Labels'] = re_1001['Label_List'].apply(lambda x: labels_to_onehot(x, label_to_num))


                                 Review Label_List OneHot_Labels
0  저도 모르게 은근 여기 자주 시켜먹고 있었네요 늘 잘먹고 있습니다    [맛, 배달]  [1, 0, 0, 1]
1                 김치찌개 된장찌개 잡채 다 잘먹었습니다        [맛]  [1, 0, 0, 0]
2    처음시켜보는데 맛있고 리뷰서비스도 괜찮네요 다음에도 또시킬듯요        [맛]  [1, 0, 0, 0]
3               맛있습니다 근데 국물이 안와서 아쉽습니다     [맛, 배달]  [1, 0, 0, 1]
4                             맛있게잘먹었습니다        [맛]  [1, 0, 0, 0]


<ipython-input-11-773fe37250da>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  re_1000['Label_List'] = re_1000['Label'].apply(string_to_list)
<ipython-input-11-773fe37250da>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  re_1000['OneHot_Labels'] = re_1000['Label_List'].apply(lambda x: labels_to_onehot(x, label_to_num))
<ipython-input-11-773fe37250da>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



In [ ]:
re_1000[['Review', 'Label_List', 'OneHot_Labels']]

,Review,Label_List,OneHot_Labels
0,저도 모르게 은근 여기 자주 시켜먹고 있었네요 늘 잘먹고 있습니다,"[맛, 배달]","[1, 0, 0, 1]"
1,김치찌개 된장찌개 잡채 다 잘먹었습니다,[맛],"[1, 0, 0, 0]"
2,처음시켜보는데 맛있고 리뷰서비스도 괜찮네요 다음에도 또시킬듯요,[맛],"[1, 0, 0, 0]"
3,맛있습니다 근데 국물이 안와서 아쉽습니다,"[맛, 배달]","[1, 0, 0, 1]"
4,맛있게잘먹었습니다,[맛],"[1, 0, 0, 0]"
...,...,...,...
395,맛있다는 소문 듣고 드디어 주문해봤어요 엽떡보다 덜 자극적이면서 구성은 비슷해서 정...,"[맛, 배달]","[1, 0, 0, 1]"
396,실속세트가있어서좋은거같아요 이렇게세트로시키니까 둘이서먹기딱좋은거같네요 남기지않고다먹...,"[맛, 양]","[1, 1, 0, 0]"
397,배달은 항상 50분 넘게걸려요 그래도 맛있으니 2번째요,"[맛, 배달]","[1, 0, 0, 1]"
398,떡깨비는 첨 시켜보는 곳인데 너무 좋아요 개인적인 요청사항이 있었어서 전화로 한번 ...,[맛],"[1, 0, 0, 0]"


In [ ]:
re_1001[['Review', 'Label_List', 'OneHot_Labels']]

,Review,Label_List,OneHot_Labels
1000,지난주에 먹었는데 또 생각나서 시켰습니다 역시나 양 많고 너무 맛있었어요 잘먹었습니다,"[맛, 양]","[1, 1, 0, 0]"
1001,오랜만에 먹는데 너무 맛있어요,[맛],"[1, 0, 0, 0]"
1002,사진이 좀 맛없게 찍혔지만 맛 괜찮았어요 그나저나 배달이 20분 정도 빨리 도착해서...,"[가격, 맛, 배달]","[1, 0, 1, 1]"
1003,에브리데이 맛난 집 1인분인데 야채만으로도 원체양이많아서 고기빼고시켜봤는데 어른이맛...,[맛],"[1, 0, 0, 0]"
1004,맛있게 잘먹었습니다,[맛],"[1, 0, 0, 0]"
...,...,...,...
1995,볶음짬뽕은 아주 맛있었어요 다만 배달도 빠른데 소고기 탕수육 바삭함이 없어 아쉬웠어오,"[맛, 배달]","[1, 0, 0, 1]"
1996,맛있는 좋은 음식 잘 먹었습니다 감사합니다 사장님,[맛],"[1, 0, 0, 0]"
1997,잘먹었습니다 짜장면 맛있었는데 사이드만두는 양이 작아요,"[맛, 양]","[1, 1, 0, 0]"
1998,맛있는 음식 빠른 배달 감사합니다,"[맛, 배달]","[1, 0, 0, 1]"


In [ ]:
import numpy as np
from transformers import AutoTokenizer

# 모델 이름 및 토크나이저 정의
model_name = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

# 주어진 데이터프레임에서 텍스트와 라벨을 인코딩하는 함수
def process_data(df, tokenizer):
    texts = df["Review"].tolist()
    labels_encoded = np.array(df["OneHot_Labels"].tolist())

    # 텍스트 인코딩
    encodings = tokenizer(texts, padding=True, truncation=True, max_length=256, return_tensors="pt")
    return encodings, labels_encoded

# 각 데이터셋에 대한 처리
train_encodings, train_labels_encoded = process_data(re_1000, tokenizer)
val_encodings, val_labels_encoded = process_data(re_1001, tokenizer)


In [ ]:
# 첫 5개의 훈련 데이터 및 라벨 출력
for i in range(5):
    print("Review:", re_1000["Review"].iloc[i])
    print("Encoded:", train_encodings['input_ids'][i])
    print("Label:", train_labels_encoded[i])
    print("-" * 50)

# 첫 5개의 검증 데이터 및 라벨 출력
for i in range(5):
    print("Review:", re_1001["Review"].iloc[i])
    print("Encoded:", val_encodings['input_ids'][i])
    print("Label:", val_labels_encoded[i])
    print("-" * 50)


Review: 저도 모르게 은근 여기 자주 시켜먹고 있었네요 늘 잘먹고 있습니다
Encoded: tensor([     1,   8345,   1248,   3709,   4851,   2318,    260,    870,   9968,
          5742,   1623,   4916,   3851,    260, 113620,  82509,   1001,  36405,
         38646,    260,  28043,  14985,  82509,   1001,   6800,      2,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,   

In [ ]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

# 데이터를 데이터셋으로 변환
train_dataset = CustomDataset(train_encodings, train_labels_encoded)
val_dataset = CustomDataset(val_encodings, val_labels_encoded)


# dataloader 생성

In [ ]:
from torch.utils.data import DataLoader

# DataLoader 설정
batch_size = 16

train_dataset = CustomDataset(train_encodings, train_labels_encoded)
val_dataset = CustomDataset(val_encodings, val_labels_encoded)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)


In [ ]:
from transformers import AutoModelForSequenceClassification
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.nn import BCEWithLogitsLoss

# 모델, 손실 함수, 옵티마이저 초기화
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4, ignore_mismatched_sizes=True).to('cuda')
loss_fn = BCEWithLogitsLoss().to('cuda')
optimizer = AdamW(model.parameters(), lr=1e-5)


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at MoritzLaurer/mDeBERTa-v3-base-mnli-xnli and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([4]) in the model instantiated
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from tqdm import tqdm

epochs = 8
total_steps = len(train_loader) * epochs
model.train()

for epoch in range(epochs):
    total_loss = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):

        # 배치 데이터를 CUDA에 할당
        inputs = batch['input_ids'].to('cuda')
        attention_mask = batch['attention_mask'].to('cuda')
        labels = batch['labels'].to('cuda')

        # 예측 및 손실 계산
        outputs = model(inputs, attention_mask=attention_mask)
        logits = outputs.logits
        loss = loss_fn(logits, labels.float())

        # 역전파
        loss.backward()

        # 가중치 업데이트
        optimizer.step()

        # 그래디언트 초기화
        optimizer.zero_grad()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Train Loss: {avg_train_loss:.4f}")


Epoch 1/8: 100%|██████████| 25/25 [00:17<00:00,  1.42it/s]


Train Loss: 0.3214


Epoch 2/8: 100%|██████████| 25/25 [00:17<00:00,  1.45it/s]


Train Loss: 0.3241


Epoch 3/8: 100%|██████████| 25/25 [00:16<00:00,  1.49it/s]


...

Train Loss: 0.3220


Epoch 7/8: 100%|██████████| 25/25 [00:16<00:00,  1.48it/s]


Train Loss: 0.3240


Epoch 8/8: 100%|██████████| 25/25 [00:16<00:00,  1.49it/s]

Train Loss: 0.3230


In [ ]:
model.save_pretrained("rere_1000")
model = AutoModelForSequenceClassification.from_pretrained("rere_1000")

# 검증데이터

In [ ]:
model = model.to('cuda')

model.eval()  # 모델을 평가 모드로 설정

total_loss = 0
correct_predictions = 0
total_predictions = 0

for batch in val_loader:
    with torch.no_grad():  # 기울기 계산을 하지 않음
        inputs = batch['input_ids'].to('cuda')
        attention_mask = batch['attention_mask'].to('cuda')
        labels = batch['labels'].to('cuda')

        outputs = model(inputs, attention_mask=attention_mask)
        logits = outputs.logits
        loss = loss_fn(logits, labels.float())
        total_loss += loss.item()

        # 로지스틱 회귀 결과를 확률로 변환
        probs = torch.sigmoid(logits)
        # 확률을 라벨 (0 or 1)로 변환
        preds = (probs > 0.5).long()

        correct_predictions += (preds == labels).sum().item()
        total_predictions += labels.numel()

avg_val_loss = total_loss / len(val_loader)
accuracy = correct_predictions / total_predictions

print(f"Validation Loss: {avg_val_loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")


Validation Loss: 0.3220
Validation Accuracy: 0.8810


# 학습 모델 예측 -> 1000~2000 데이터

In [ ]:
result_re1001 = result_df["Review"][1000:2000]

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

result_re1001 = result_df["Review"][1000:2000].tolist()

inputs = tokenizer(result_re1001, truncation=True, padding=True, return_tensors="pt")

# 데이터셋 데이터 로더 설정
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'])
data_loader = DataLoader(dataset, batch_size=16, shuffle=False)

# 모델 예측
all_preds = []
with torch.no_grad():
    for batch in data_loader:
        input_ids, attention_mask = (item.to('cuda') for item in batch)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.sigmoid(logits)
        preds = (probs > 0.5).long()  # 확률을 라벨 (0 or 1)로 변환
        all_preds.extend(preds.cpu().tolist())

print(all_preds)


[[1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.utils.data import DataLoader, TensorDataset

# 레이블을 숫자로 변환하는 딕셔너리
label_to_num = {'맛': 0, '양': 1, '가격': 2, '배달': 3}

model = AutoModelForSequenceClassification.from_pretrained("rere_1000").to('cuda')

result_re1001 = result_df["Review"][1000:2000].tolist()

inputs = tokenizer(result_re1001, truncation=True, padding=True, return_tensors="pt")

# 데이터셋 데이터 로더 설정
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'])
data_loader = DataLoader(dataset, batch_size=16, shuffle=False)

# 모델 예측
all_preds = []
with torch.no_grad():
    for batch in data_loader:
        input_ids, attention_mask = (item.to('cuda') for item in batch)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.sigmoid(logits)
        preds = (probs > 0.5).long()  # 확률을 라벨 (0 or 1)로 변환
        all_preds.extend(preds.cpu().tolist())

# 인코딩된 라벨을 텍스트 라벨로 변환하는 함수
def one_hot_to_labels(one_hot_labels, label_to_num):
    num_to_label = {v: k for k, v in label_to_num.items()}
    labels_list = []
    for one_hot in one_hot_labels:
        labels = [num_to_label[i] for i, value in enumerate(one_hot) if value == 1]
        labels_list.append(labels)
    return labels_list

# 인코딩된 라벨 텍스트 라벨로 변환
predicted_label_list = one_hot_to_labels(all_preds, label_to_num)

# re_1001 데이터프레임에 예측된 라벨 추가
result_df["Predicted_Labels"] = [""] * 1000 + predicted_label_list + [""] * (len(result_df) - 2000)

# 결과 출력
print(result_df[['Review', 'Predicted_Labels']].iloc[1000:1010])


                                                 Review Predicted_Labels
1000    지난주에 먹었는데 또 생각나서 시켰습니다 역시나 양 많고 너무 맛있었어요 잘먹었습니다              [맛]
1001                                  오랜만에 먹는데 너무 맛있어요               [맛]
1002  사진이 좀 맛없게 찍혔지만 맛 괜찮았어요 그나저나 배달이 20분 정도 빨리 도착해서...              [맛]
1003  에브리데이 맛난 집 1인분인데 야채만으로도 원체양이많아서 고기빼고시켜봤는데 어른이맛...              [맛]
1004                                         맛있게 잘먹었습니다              [맛]
1005  마라탕순한맛이랑 꽃빵시켰는데 생각보다 안매워서 괜찮았고 좀 싱거운 느낌이 있긴한데 ...              [맛]
1006                                      무조건 여기에서 먹습니다              [맛]
1007                  애들이 너무 좋아해요 다른데보다 여기가 더맛있어요 맛집 인정              [맛]
1008  음 전 가게에 누를 끼치고싶진않습니다 그래서 별점5개 다 드려요 맛도있구요 워낙 블...              [맛]
1009                                   너무너무 맛나게 잘 먹었어요               [맛]


In [ ]:
result_df[['Review', 'Predicted_Labels']].iloc[1000:2000]

,Review,Predicted_Labels
1000,지난주에 먹었는데 또 생각나서 시켰습니다 역시나 양 많고 너무 맛있었어요 잘먹었습니다,[맛]
1001,오랜만에 먹는데 너무 맛있어요,[맛]
1002,사진이 좀 맛없게 찍혔지만 맛 괜찮았어요 그나저나 배달이 20분 정도 빨리 도착해서...,[맛]
1003,에브리데이 맛난 집 1인분인데 야채만으로도 원체양이많아서 고기빼고시켜봤는데 어른이맛...,[맛]
1004,맛있게 잘먹었습니다,[맛]
...,...,...
1995,볶음짬뽕은 아주 맛있었어요 다만 배달도 빠른데 소고기 탕수육 바삭함이 없어 아쉬웠어오,[맛]
1996,맛있는 좋은 음식 잘 먹었습니다 감사합니다 사장님,[맛]
1997,잘먹었습니다 짜장면 맛있었는데 사이드만두는 양이 작아요,[맛]
1998,맛있는 음식 빠른 배달 감사합니다,[맛]
